In [13]:
import pickle#引入pickle库 实现文本文件与二进制文件之间转化
#pt1 转换数据
data=[]#定义data列表 以便后面保存嵌套列表
with open("ML100k.data","r") as file:
    for line in file:#对数据逐行迭代
        u_id,f_id,rate,time = map(int,line.strip().split())#将每一行的字符串去除杂乱符号并分割 分割后分别赋值给四个变量
        #尝试使用新学会的strip和split默认参数用法：不传参则去掉默认符号，按字符串分割
        data.append([u_id,f_id,rate,time])#将四个变量作为一维数组传入data数组中，作为第一条数据（data可视为二维数组）
print(f"前10条：{data[:10]}")#测试数据

with open("data.pickle","wb")as file:
    pickle.dump(data,file)#将处理好的数据转化为二进制存入pickle文件

#pt2 过滤数据 （目标：1/删除少于五个电影评分的不活跃用户 2/删除被少于5个用户评分的冷门电影）
def filt(data,key=5):
    while(True):
        #统计用户评价次数和电影被评价次数
        user_count ={}#通过字典（key-value对应 用户-对应值）模拟计数器
        film_count ={}
        for u_id,f_id,rate,time in data:#对data中的每行数据迭代 并拆包赋值给四个变量
            user_count[u_id] = user_count.get(u_id,0)+1#如果user_count存在u_id这个key，则更新value为对应的value+1
            film_count[f_id] = film_count.get(f_id,0)+1#逻辑同上
        new_data =[]#定义新列表，记录统计之后的新数据
        #筛选数据(逐行迭代，如果用户评分次数和电影被评分次数同时大于key则存入新数据)
        for u_id,f_id,rate,time in data:
            if user_count[u_id] >=key and film_count[f_id] >=key:
                new_data.append([u_id,f_id,rate,time])
        #这里会发现存在一个很难受的bug：当某些数据被删除后，会导致某些电影的被评价次数降低，这样的话依旧不符合要求；同理也会造成某些用户评价次数降低
        #所以需要不断迭代，直到新数据和处理前数据相同：表明数据已经稳定
        if len(new_data) ==len(data):
            break;
        data = new_data
    return data

#对我们的数据使用我们定义的函数进行处理
data =filt(data,5)
print(f"过滤后的数据共有：{len(data)}条")

#pt3 统计数据
#统计指标：
#一 1/用户数量 2/电影数量 3/评分数量 二 1/稀疏度：评分数量/（用户数*电影数）） 
#三 1/某用户的平均评分(从小到大排序) 2/某个电影的平均评分（从小到大排序） 3/全部评分的平均评分
#四 统计所有评分中1-5分的分布情况（每个分数有多少条评分记录）
#五 将用户和电影分别重新从0标号
user = sorted(set(u_id for u_id,_,_,_ in data))# set是无序列表，可以去重 通过sorted函数对去重后的id进行升序排序
film = sorted(set(f_id for _,f_id,_,_ in data))
rating= [rate for _,_,rate,_ in data]#评分并不需要去重
u_sum = len(user)
f_sum = len(film)
r_sum = len(rating)
sparsity = r_sum/(f_sum*u_sum)
print(f"用户数量为：{u_sum} 电影数量为：{f_sum} 评分数量为：{r_sum} 可见 稀疏度为：{sparsity}")
#第一二部分处理完成
u_avgr ={}
u_count ={}
f_avgr ={}
f_count ={}
r_count =0
# for u in user:#使用去重后的用户列表进行迭代
#     for u_id,_,r,_ in data: 
#         if u_id ==u:
#             u_count[u] =u_count.get(u,0)+1
#     u_avgr[u] = [(u_avgr.get(u,0)+r) for u_id,_,r,_ in data if u_id ==u]#对data中的数据逐行迭代，如果用户_id和我们迭代的u相同，则将将u的value+r
# for f in film:
#     for _,f_id,r,_ in data: 
#         if f_id ==f:
#             f_count[f] =f_count.get(f,0)+1
#     f_avgr[f] = [(f_avgr.get(f,0)+r) for _,f_id,r,_ in data if f_id ==f]

# u_avgr={key:u_avgr.get(key,0)/u_count[key] for key in u_avgr}
# f_avgr={key:f_avgr.get(key,0)/f_count[key] for key in f_avgr}
# !上述代码是失败经历 出现严重逻辑错误：get(u,0)每次都返回默认值0而不是将u的value赋值为0
# 用户平均评分
for u_id,_,rate,_ in data:
    if u_id not in u_avgr:
        u_avgr[u_id] = 0
        u_count[u_id] = 0
    u_avgr[u_id]+= rate
    u_count[u_id]+= 1
u_avgr ={u: u_avgr[u] / u_count[u] for u in u_avgr}
print(f"前十条用户平均评分为：{list(u_avgr.items())[:10]}")
# 电影平均评分
for _,f_id,rate,_ in data:
    if f_id not in f_avgr:
        f_avgr[f_id] = 0
        f_count[f_id] = 0
    f_avgr[f_id] += rate
    f_count[f_id] += 1
f_avgr = {f: f_avgr[f] / f_count[f] for f in f_avgr}
print(f"前十条电影平均评分为：{list(f_avgr.items())[:10]}")
for r in rating:
    r_count+=r
r_avg=r_count/r_sum
print(f"平均评分为：{r_avg}")
#第三部分处理完成
rate_level={1:0,2:0,3:0,4:0,5:0}
for r in rating:
    rate_level[r]=rate_level.get(r,0)+1
print(f"电影层次结果：{rate_level}")
#第四部分处理完成
user_map = {}
film_map = {}
new_u_count = 0
new_f_count = 0
for u in user:
    user_map[u]=new_u_count
    new_u_count+=1
for f in film:
    film_map[f]=new_f_count
    new_f_count+=1
print(f"第一个用户是：{user_map[user[0]]}最后一个用户是：{user_map[user[len(user)-1]]}")
print(f"第一个电影是：{film_map[film[0]]}最后一个电影是：{film_map[film[len(film)-1]]}")
#第五部分处理完成

前10条：[[196, 242, 3, 881250949], [186, 302, 3, 891717742], [22, 377, 1, 878887116], [244, 51, 2, 880606923], [166, 346, 1, 886397596], [298, 474, 4, 884182806], [115, 265, 2, 881171488], [253, 465, 5, 891628467], [305, 451, 3, 886324817], [6, 86, 3, 883603013]]
过滤后的数据共有：99287条
用户数量为：943 电影数量为：1349 评分数量为：99287 可见 稀疏度为：0.07804925214624242
前十条用户平均评分为：[(196, 3.6153846153846154), (186, 3.4130434782608696), (22, 3.3515625), (244, 3.6455696202531644), (166, 3.55), (298, 4.031496062992126), (115, 3.9347826086956523), (253, 3.979381443298969), (305, 3.415525114155251), (6, 3.6350710900473935)]
前十条电影平均评分为：[(242, 3.9914529914529915), (302, 4.161616161616162), (377, 2.1538461538461537), (51, 3.45679012345679), (346, 3.642857142857143), (474, 4.252577319587629), (265, 3.8634361233480177), (465, 3.5647058823529414), (451, 3.347058823529412), (86, 3.94)]
平均评分为：3.536182984680774
电影层次结果：{1: 5900, 2: 11259, 3: 26963, 4: 34035, 5: 21130}
第一个用户是：0最后一个用户是：942
第一个电影是：0最后一个电影是：1348
